**Frame to find best match fragment**

Activities of Shadow Play

In [1]:
!pip install --upgrade moviepy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 388 kB 32.4 MB/s 
     |████████████████████████████████| 26.9 MB 1.5 MB/s 
  Created wheel for moviepy: filename=moviepy-1.0.3-py3-none-any.whl size=110742 sha256=6e076933470d105bef0fb2837f559054a57150d478b8e64d8bf5b8c37ea1fb45
  Stored in directory: /root/.cache/pip/wheels/e4/a4/db/0368d3a04033da662e13926594b3a8cf1aa4ffeefe570cfac1
Successfully built moviepy
  Attempting uninstall: moviepy
    Found existing installation: moviepy 0.2.3.5
    Uninstalling moviepy-0.2.3.5:
      Successfully uninstalled moviepy-0.2.3.5


In [2]:
! git clone https://github.com/kenshohara/video-classification-3d-cnn-pytorch.git

Cloning into 'video-classification-3d-cnn-pytorch'...
remote: Enumerating objects: 121, done.
remote: Total 121 (delta 0), reused 0 (delta 0), pack-reused 121
Receiving objects: 100% (121/121), 158.63 KiB | 19.83 MiB/s, done.
Resolving deltas: 100% (63/63), done.


In [3]:
%cd video-classification-3d-cnn-pytorch/
!pwd

/content/video-classification-3d-cnn-pytorch
/content/video-classification-3d-cnn-pytorch


In [4]:
import json
import os
from moviepy.editor import *
from moviepy import *

In [5]:
def LoadVideosToFile(folder):
  videos = os.listdir(folder)
  with open('input', 'w') as inputfile:
    for v in videos:
      if v.split('.')[-1] == 'mp4':
        inputfile.write(v+'\n')
  return videos

In [ ]:
LoadVideosToFile('/content/drive/MyDrive/SkillClass/Video/VideoToProcess')

['chineseshadowplay.mp4']

In [ ]:
! python main.py --input input --video_root /content/drive/MyDrive/SkillClass/Video/VideoToProcess --output /content/drive/MyDrive/SkillClass/Video/VideoOut/output.json --model /content/drive/MyDrive/SkillClass/resnet-34-kinetics-cpu.pth --model_depth 34  --no_cuda --mode score --resnet_shortcut A

loading model /content/drive/MyDrive/SkillClass/resnet-34-kinetics-cpu.pth
/content/drive/MyDrive/SkillClass/Video/VideoToProcess/chineseshadowplay.mp4
ffmpeg version 3.4.11-0ubuntu0.1 Copyright (c) 2000-2022 the FFmpeg developers
  built with gcc 7 (Ubuntu 7.5.0-3ubuntu1~18.04)
  configuration: --prefix=/usr --extra-version=0ubuntu0.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --enable-gpl --disable-stripping --enable-avresample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librubberband --enable-librsvg --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libssh --enable-libtheora --

In [ ]:
! python generate_result_video/generate_result_video.py /content/drive/MyDrive/SkillClass/Video/VideoOut/output.json /content/drive/MyDrive/SkillClass/Video/VideoToProcess /content/drive/MyDrive/SkillClass/Video/Video_pred class_names_list 5

/content/drive/MyDrive/SkillClass/Video/VideoToProcess/chineseshadowplay.mp4
ffmpeg version 3.4.11-0ubuntu0.1 Copyright (c) 2000-2022 the FFmpeg developers
  built with gcc 7 (Ubuntu 7.5.0-3ubuntu1~18.04)
  configuration: --prefix=/usr --extra-version=0ubuntu0.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --enable-gpl --disable-stripping --enable-avresample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librubberband --enable-librsvg --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvorbis --enable-libvpx --enable-libwavpack --

searching through multiple videos at once for segments that match a certain input **video**

In [6]:
%cd video-classification-3d-cnn-pytorch/

[Errno 2] No such file or directory: 'video-classification-3d-cnn-pytorch/'
/content/video-classification-3d-cnn-pytorch


In [7]:
import json
import os
from moviepy.editor import *
from moviepy import *
import shutil
import numpy as np

In [8]:
def loadVideosToInfile(folder):
    videos = os.listdir(folder)
    with open('input', 'w') as inputfile:
      for v in videos:
        if v.split('.')[-1] == 'mp4':
          inputfile.write(v+'\n')
    return videos

In [9]:
def readAllOutput(outputFolder):
    dictionaries = {}
    output_files = os.listdir(outputFolder)
    for f in output_files:
        if f.split('.')[-1]=='json':
            file = open(os.path.join(outputFolder,f))
            segments = json.load(file)
            for s in segments:
                dictionaries[s['video']] = s
    
    return dictionaries

In [10]:
def searchForMatch(keyFilmName, featureDictionary):
    bestRangeStart = 0
    bestRangeFilmName = ''
    bestRangeScore = 1000000000
    
    keyClips = featureDictionary[keyFilmName]['clips']
    keyFeatures = []
    for c in keyClips:
        keyFeatures.append(np.array(c['features']))
    
    for film in featureDictionary.keys():
        if not film == keyFilmName:
            filmClips = featureDictionary[film]['clips']
            filmFeatures = []
            for c in filmClips:
                filmFeatures.append(np.array(c['features']))
        
            for i in range(len(filmFeatures)-len(keyFeatures)):
                distance = 0
                for j in range(len(keyFeatures)):
                    d = np.linalg.norm(filmFeatures[i+j]-keyFeatures[j])
                    distance += d
                if distance < bestRangeScore:
                    bestRangeScore = distance
                    bestRangeStart = i
                    bestRangeFilmName = film
    
    matchingFragment = [bestRangeFilmName, bestRangeStart, len(keyFeatures)]
    
    return matchingFragment

In [11]:
def extractFragment(fragment, filmFolder):
    filmFileName = fragment[0]
    start = fragment[1]
    nrClips = fragment[2]
    
    film = VideoFileClip(os.path.join(filmFolder, filmFileName))
    tsfps = film.fps
    tsFrameLength = 1/tsfps

    nrFrames = nrClips*16
    startFrame = start*16

    startTime = startFrame*tsFrameLength
    endTime = startTime+nrFrames*tsFrameLength

    filmFragment = film.subclip(startTime,endTime)
    
    return filmFragment

In [ ]:
loadVideosToInfile('/content/drive/MyDrive/SkillClass/Video/VideoProcessed')

['frame-169129.mp4',
 'frame-107229.mp4',
 'frame-113494.mp4',
 'frame-067277.mp4',
 'frame-145683.mp4',
 'frame-005394.mp4',
 'frame-070240.mp4',
 'frame-137865.mp4',
 'frame-032893.mp4',
 'frame-134436.mp4',
 'frame-015515.mp4',
 'frame-000713.mp4',
 'frame-131648.mp4',
 'frame-042416.mp4',
 'frame-039868.mp4',
 'frame-011298.mp4',
 'frame-004321.mp4',
 'frame-002669.mp4',
 'frame-113631.mp4',
 'frame-101817.mp4',
 'frame-011772.mp4',
 'frame-123643.mp4',
 'frame-012181.mp4',
 'frame-045420.mp4',
 'frame-068917.mp4',
 'frame-135233.mp4',
 'frame-010360.mp4',
 'frame-027949.mp4',
 'frame-083093.mp4',
 'frame-020438.mp4',
 'frame-051287.mp4',
 'frame-041239.mp4',
 'frame-049296.mp4',
 'frame-063743.mp4',
 'frame-010053.mp4',
 'frame-140698.mp4',
 'frame-056822.mp4',
 'frame-023213.mp4',
 'frame-045411.mp4',
 'frame-023254.mp4',
 'frame-095208.mp4',
 'frame-068121.mp4',
 'frame-123364.mp4',
 'frame-039109.mp4',
 'frame-094928.mp4',
 'frame-200221.mp4',
 'frame-156380.mp4',
 'frame-15584

In [ ]:
# run the model to process all films currently in the input_films folder, and save it to a json file, here named by the date

!python main.py --input input --video_root /content/drive/MyDrive/SkillClass/Video/VideoProcessed --output /content/drive/MyDrive/SkillClass/Video/output/output221212.json --model /content/drive/MyDrive/SkillClass/resnet-34-kinetics-cpu.pth --mode feature --no_cuda

Streaming output truncated to the last 5000 lines.
  libavfilter     6.107.100 /  6.107.100
  libavresample   3.  7.  0 /  3.  7.  0
  libswscale      4.  8.100 /  4.  8.100
  libswresample   2.  9.100 /  2.  9.100
  libpostproc    54.  7.100 / 54.  7.100
Input #0, mov,mp4,m4a,3gp,3g2,mj2, from '/content/drive/MyDrive/SkillClass/Video/VideoProcessed/frame-113631.mp4':
  Metadata:
    major_brand     : isom
    minor_version   : 512
    compatible_brands: isomiso2avc1mp41
    encoder         : Lavf58.20.100
  Duration: 00:00:31.08, start: -0.007333, bitrate: 1769 kb/s
    Stream #0:0(und): Video: h264 (High) (avc1 / 0x31637661), yuv420p, 1280x688 [SAR 1:1 DAR 80:43], 1635 kb/s, 23.98 fps, 23.98 tbr, 24k tbn, 47.95 tbc (default)
    Metadata:
      handler_name    : VideoHandler
    Stream #0:1(und): Audio: aac (LC) (mp4a / 0x6134706D), 48000 Hz, stereo, fltp, 128 kb/s (default)
    Metadata:
      handler_name    : SoundHandler
Stream mapping:
  Stream #0:0 -> #0:0 (h264 (native) -> mjp

In [237]:
loadVideosToInfile('/content/drive/MyDrive/SkillClass/Video/VideoToProcess')

['52lunge.mp4']

In [238]:
# run the model to process all films currently in the input_films folder, and save it to a json file, here named by the date

!python main.py --input input --video_root /content/drive/MyDrive/SkillClass/Video/VideoToProcess --output /content/drive/MyDrive/SkillClass/Video/output/output.json --model /content/drive/MyDrive/SkillClass/resnet-34-kinetics-cpu.pth --mode feature --no_cuda

loading model /content/drive/MyDrive/SkillClass/resnet-34-kinetics-cpu.pth
/content/drive/MyDrive/SkillClass/Video/VideoToProcess/52lunge.mp4
ffmpeg version 3.4.11-0ubuntu0.1 Copyright (c) 2000-2022 the FFmpeg developers
  built with gcc 7 (Ubuntu 7.5.0-3ubuntu1~18.04)
  configuration: --prefix=/usr --extra-version=0ubuntu0.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --enable-gpl --disable-stripping --enable-avresample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librubberband --enable-librsvg --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libssh --enable-libtheora --enable-lib

In [239]:
dicts = readAllOutput('/content/drive/MyDrive/SkillClass/Video/output')

In [240]:
fragment = searchForMatch('52lunge.mp4', dicts)

In [241]:
fragmentFilm = extractFragment(fragment, '/content/drive/MyDrive/SkillClass/Video/VideoProcessed')

In [242]:
fragmentFilm.ipython_display()

Moviepy - Building video __temp__.mp4.
MoviePy - Writing audio in __temp__TEMP_MPY_wvf_snd.mp3


MoviePy - Done.
Moviepy - Writing video __temp__.mp4



Moviepy - Done !
Moviepy - video ready __temp__.mp4


In [243]:
fragmentFilm.write_videofile("/content/drive/MyDrive/SkillClass/Video/VideoOut/52.mp4", audio_codec='aac')

Moviepy - Building video /content/drive/MyDrive/SkillClass/Video/VideoOut/52.mp4.
MoviePy - Writing audio in 52TEMP_MPY_wvf_snd.mp4


MoviePy - Done.
Moviepy - Writing video /content/drive/MyDrive/SkillClass/Video/VideoOut/52.mp4



Moviepy - Done !
Moviepy - video ready /content/drive/MyDrive/SkillClass/Video/VideoOut/52.mp4
